# django-tables2
테이블의 컬럼 데이터 수정하기
https://django-tables2.readthedocs.io/en/latest/pages/custom-data.html
1. 우선 작업 Process 를 잘 익히자
1. Data 호출 -> **Data 를 클래스 인스턴스로** 변환 
1. **django-table2 사용자함수를** view에 정의
1. Data 클래스 인스턴스를 **django-table2 사용자함수에** 적용결과를 template로 전달

## 1 테이블 객체 접근자
Accessors [link](https://django-tables2.readthedocs.io/en/latest/pages/custom-data.html)
1. 하지만 이거 없어도 dict 객체접근 방식을 사용가능

In [8]:
from django_tables2 import A
# List dict 데이터를 수정하기
data = {'abc': {
    'one':{
        'two':'three'
    }
}}

A('abc').resolve(data)

{'one': {'two': 'three'}}

In [9]:
A('abc.one.two').resolve(data)

'three'

## 2 테이블 컬럼의 속성변경
Table.render_**컬럼명** 메소드
1. 위의 **컬럼명**은 **Table의 컬럼명과 일치**해야 한다
1. 컬럼별 소숫점 처리등 다양한 처리가 가능하다
1. 단 연산컬럼에 **empty value** 가 중간에 있으면 최초값으로 처리하므로 주의 (False 등은 자동처리됨)

In [11]:
import django_tables2 as tables
import itertools # 자신만의 반복자

# 1 Data Instance Function : 인덱스 조작기능 추가
class SimpleTable(tables.Table):
    
    row_number = tables.Column(empty_values=()) # 객체의 갯수값 index 값 
    id_num  = tables.Column()
    age     = tables.Column()
    pay     = tables.Column()
    
    def __init__(self, *args, **kwargs): 
        super(SimpleTable, self).__init__(*args, **kwargs)
        self.counter = itertools.count()
    # 반복자 갯수 Count
    def render_row_number(self):
        return 'Row %d' % next(self.counter) 
    # id 컬럼의 값에 < > 씌우기
    def render_id_num(self, value):  
        return '<%s>' % value
    # 1,000 콤마 추가하기
    def render_pay(self, value):
        return "{:,}".format(value)

In [12]:
# 2 Data Set
data_set = [
    {'age': 31, 'id_num': 10, 'pay':False}, 
    {'age': 34, 'id_num': 11, 'pay':232123},
    {'age': 41, 'id_num': 13, 'pay':523123},
    {'age': 39, 'id_num': 18, 'pay':928123},
    {'age': 32, 'id_num': 91, 'pay':132123},
]
# 3 Apply the DataSet
table = SimpleTable(data_set)

for i in range(3):
    print(', '.join(map(str, table.rows[i]))) # 행번호별 객체값을 관리 가능하다

Row 0, <10>, 31, 0
Row 1, <11>, 34, 232,123
Row 2, <13>, 41, 523,123


## SubClassing

In [15]:
import django_tables2 as tables
class UpperColumn(tables.Column):
    def render(self, value):
        return value.upper()

class Example(tables.Table):
    normal = tables.Column()
    upper = UpperColumn()

data = [{'normal': 'Hi there!',
         'upper':  'Hi there!'}]
# renders to something like this:
table = Example(data)
table

In [16]:
# 파일이름만 호출하고, 태그를 추가한 탬플릿을 생성 가능하다
from django.utils.html import format_html
class ImageColumn(tables.Column):
    def render(self, value):
        return format_html('<img src="/media/img/{}.jpg" />', value)

In [22]:
ImageColumn(data).value

# 랜더링 결과
# '''<table>
#     <thead><tr><th>Normal</th><th>Upper</th></tr></thead>
#     <tbody><tr><td>Hi there!</td><td>HI THERE!</td></tr></tbody>
# </table>'''

<bound method Column.value of <__main__.ImageColumn object at 0x7f3064353198>>

## 정렬 순서 바꾸기
SubClassing

In [8]:
# # Django / App/ models.py
# class Person(models.Model):
#     first_name  = models.CharField(max_length=200)
#     family_name = models.CharField(max_length=200)

#     @property
#     def name(self):
#         return '{} {}'.format(self.first_name, self.family_name)
# #---------------------------------------------------------------
# # tables.py
# class PersonTable(tables.Table):
#     name = tables.Column()

In [26]:
data_set = [
    {'first_name': 'YB', 'family_name': 'Kim'}, 
    {'first_name': 'KB', 'family_name': 'Lee'}, 
    {'first_name': 'AB', 'family_name': 'Lee'}, 
    {'first_name': 'CB', 'family_name': 'Kim'}, 
]

In [27]:
# 1. Data Instance Function
import django_tables2 as tables

class PersonTable(tables.Table):
    first_name  = tables.Column()
    family_name = tables.Column()
    # 해당컬럼 정렬시 우선적 정렬기준 컬럼을 지정 가능하다
    summary    = tables.Column(order_by=("name", "population"))
    
    @property
    def name(self):
        return '{} {}'.format(self.first_name, self.family_name)

In [28]:
table = PersonTable(data_set)
table.order_by = 'name'
table

## 테이블에 CSS id값 추가하기
자세한 내용 보기 : https://django-tables2.readthedocs.io/en/latest/pages/column-attributes.html

In [12]:
import django_tables2 as tables

class SimpleTable(tables.Table):
    name = tables.Column(attrs={'th': {'id': 'foo'}})

# >>> 위 내용으로 랜더링 결과
# '{snip}<thead><tr><th id="foo">{snip}<tbody><tr><td>{snip}'

In [13]:
# id값 만이 아니라 연산결과 값의 추가도 가능하다
class Table(tables.Table):
    person = tables.Column(attrs={
        'td': {'data-length': lambda value: len(value)}
    })

## 테이블 Header, Footer 변경하기

accessor : 컬럼명이 다를 때 연결자 (굳이 필요한가?)

In [14]:
from django.db import models

# class Region(models.Model):
#     name = models.CharField(max_length=200)

# class Person(models.Model):
#     first_name = models.CharField(verbose_name='model verbose name', max_length=200)
#     last_name  = models.CharField(max_length=200)
#     region     = models.ForeignKey('Region')

class PersonTable(tables.Table):
    first_name = tables.Column()
    ln         = tables.Column(accessor='last_name')
    region_name = tables.Column(accessor='region.name')

# table = PersonTable(Person.objects.all())
# table.columns['first_name'].header
# #'Model Verbose Name'
# table.columns['ln'].header
# #'Last Name'
# table.columns['region_name'].header
# #'Name'

## 테이블간 데이터 교차적용
다른 테이블 인스턴스에서 데이터 불러오기

In [15]:
# 상속받은 부모컬럼명 출력
class UselessMixin(object):
    first_name = tables.Column()
    
class TestTable(UselessMixin, tables.Table):
    name = tables.Column()

TestTable.base_columns.keys()

odict_keys(['name'])

In [16]:
# 상속받은 부모와 자식컬럼명 출력
class UsefulMixin(tables.Table):
    first_name = tables.Column()

class TestTable(UsefulMixin, tables.Table):
    name = tables.Column()

TestTable.base_columns.keys()

odict_keys(['first_name', 'name'])

## CSS 스타일 적용하기
https://django-tables2.readthedocs.io/en/latest/pages/custom-rendering.html

In [17]:
import django_tables2 as tables
class SimpleTable(tables.Table):
    
    id  = tables.Column(attrs={'td': {'class': 'my-class'}})
    age = tables.Column(attrs={'tf': {'bgcolor': 'red'}})

    class Meta:
        attrs = {'class': 'mytable'}

render = """<table class="mytable">
        <tr class="odd">
            <td class="id">...</td>
            <td class="age">...</td>
        </tr>
        <tr class="even">
          <td class="id">...</td>
          <td class="age">...</td>
        </tr> </table>"""
# table = SimpleTable()

## Export TableData
다양한 포맷으로 출력하기

In [23]:
# ! pip install tablib

In [24]:
import django_tables2 as tables
from django_tables2.export.views import ExportMixin

# from .models import Person
# from .tables import MyTable

# class TableView(ExportMixin, tables.SingleTableView):
#     table_class = MyTable   # 테이블 인스턴스
#     model = Person          # models.py 모델 클래스
#     template_name = 'django_tables2/bootstrap.html'

In [ ]:
from django_tables2.config import RequestConfig
from django_tables2.export.export import TableExport

# from .models import Person
# from .tables import MyTable

def table_view(request):
    table = MyTable(Person.objects.all())
    RequestConfig(request).configure(table)
    export_format = request.GET.get('_export', None)

    if TableExport.is_valid_format(export_format):
        exporter = TableExport(export_format, table)
        return exporter.response('table.{}'.format(export_format))

    return render(request, 'table.html', {
        'table': table
    })